DS-630 Assignment 4 


2.	Train an SVM classifier on the MNIST dataset. Since SVM classifiers are binary classifiers, you will need to use one-versus-the-rest to classify all 10 digits. You may want to tune the hyperparameters using small validation sets to speed up the process. What accuracy can you reach?

Loading the dataset.

In [1]:
import numpy as np
from sklearn.datasets import fetch_openml

# Loadding the dataset 
mnist = fetch_openml('mnist_784', version=1, cache=True, as_frame=False)

Splitting into training and testing datasets set. people usually just take the first 60,000 instances for the training set and the last 10,000 instances for the test set.

In [4]:
X = mnist["data"]
y = mnist["target"].astype(np.uint8)

X_train = X[:60000]
y_train = y[:60000]
X_test = X[60000:]
y_test = y[60000:]

It's common practice to shuffle the training instances first because many training algorithms are sensitive to their order. However, since the dataset has already been scrambled, we do not need to. Start off easy by using a linear SVM classifier. There is nothing additional we need to do because it will automatically utilize the One-vs-All (also known as One-vs-the-Rest, OvR) strategy. Easy!

In [9]:
from sklearn.svm import LinearSVC

lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_train, y_train)

C:\Users\Krupesh\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(random_state=42)

Making predictions and finding accuracy

In [10]:
from sklearn.metrics import accuracy_score

y_pred = lin_clf.predict(X_train)
accuracy_score(y_train, y_pred)

0.8348666666666666

Linear model is certainly too simple for most. for MNIST 83% accuracy is too low.

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float32))
X_test_scaled = scaler.transform(X_test.astype(np.float32))

In [13]:
lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_train_scaled, y_train)

C:\Users\Krupesh\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(random_state=42)

In [14]:
y_pred = lin_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.9214

this accuracy is better than linear but still not good enough for MNIST.

In [15]:
from sklearn.svm import SVC
svm_clf = SVC(gamma="scale")
svm_clf.fit(X_train_scaled[:10000], y_train[:10000])

SVC()

In [16]:
y_pred = svm_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.9455333333333333

That's encouraging because performance has improved despite the model having been trained on six times less data. Let's perform a randomized search using cross validation to adjust the hyperparameters. To expedite the procedure, we'll perform this on a small dataset:

In [17]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(svm_clf, param_distributions, n_iter=10, verbose=2, cv=3)
rnd_search_cv.fit(X_train_scaled[:1000], y_train[:1000])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END .....C=6.855833897593921, gamma=0.04734878468163205; total time=   0.3s
[CV] END .....C=6.855833897593921, gamma=0.04734878468163205; total time=   0.2s
[CV] END .....C=6.855833897593921, gamma=0.04734878468163205; total time=   0.3s
[CV] END ....C=8.357458637398308, gamma=0.006749170096626634; total time=   0.2s
[CV] END ....C=8.357458637398308, gamma=0.006749170096626634; total time=   0.6s
[CV] END ....C=8.357458637398308, gamma=0.006749170096626634; total time=   0.6s
[CV] END ....C=9.951288087071758, gamma=0.024591191168513864; total time=   0.4s
[CV] END ....C=9.951288087071758, gamma=0.024591191168513864; total time=   0.4s
[CV] END ....C=9.951288087071758, gamma=0.024591191168513864; total time=   0.4s
[CV] END ....C=7.326066792795047, gamma=0.006331795414932815; total time=   0.5s
[CV] END ....C=7.326066792795047, gamma=0.006331795414932815; total time=   0.5s
[CV] END ....C=7.326066792795047, gamma=0.006331

RandomizedSearchCV(cv=3, estimator=SVC(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000016E4F8C2D10>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000016E4F8C2E90>},
                   verbose=2)

In [18]:
rnd_search_cv.best_estimator_

SVC(C=7.326066792795047, gamma=0.006331795414932815)

In [19]:
rnd_search_cv.best_score_

0.7099644554734376

Although it seems low, keep in mind that we only trained the model on 1,000 occasions. Retrained on the entire training set, let's use the best estimator:

In [20]:
rnd_search_cv.best_estimator_.fit(X_train_scaled, y_train)

SVC(C=7.326066792795047, gamma=0.006331795414932815)

In [21]:
y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

1.0

This is better than another model. but looks like over fitting model.

In [22]:
y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.9286

Testing on the test set using random search. It’s not bad but it looks like the model is slightly overfitting.